In [4]:
import pandas as pd
data_json = pd.read_json(r'C:\Users\razor\Desktop\NLP Assignment\NLPoutput.json', lines = True)

In [6]:
data_json.iloc[1].annotation

[{'label': ['Name'],
  'points': [{'start': 217, 'end': 229, 'text': 'Jia Qing Poh'}]},
 {'label': ['CollegeName'],
  'points': [{'start': 453, 'end': 475, 'text': 'Heriot-Watt University'}]},
 {'label': ['GraduationYear'],
  'points': [{'start': 538, 'end': 542, 'text': '2022'}]},
 {'label': ['YearsofExperience'],
  'points': [{'start': 372, 'end': 380, 'text': '4 months'}]},
 {'label': ['CompaniesWorkAt'],
  'points': [{'start': 308, 'end': 327, 'text': 'AXA Affin Insurance'}]},
 {'label': ['Email'],
  'points': [{'start': 8,
    'end': 51,
    'text': 'www.linkedin.com/in/jia-qing- poh-21a816190'}]},
 {'label': ['Skills'],
  'points': [{'start': 74,
    'end': 123,
    'text': 'Critical Thinking Problem Solving Microsoft Excel'}]}]

In [7]:
data_json["content"] = data_json["content"].apply(lambda x: x.replace("\n", " "))
data_json["content"].head(5)

0    Contact www.linkedin.com/in/ming-kit-tan- b4a9...
1    Contact www.linkedin.com/in/jia-qing- poh-21a8...
2    Contact www.linkedin.com/in/md-sariful- islam-...
3    Contact www.linkedin.com/in/binxuankong (Linke...
4    Contact www.linkedin.com/in/mohamsup (LinkedIn...
Name: content, dtype: object

In [8]:
# import logging
import json
import re

# JSON formatting functions
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    training_data = []
    lines=[]
    with open(dataturks_JSON_FilePath, 'r') as f:
        lines = f.readlines()

    for line in lines:
        data = json.loads(line)
        text = data['content'].replace("\n", " ")
        entities = []
        data_annotations = data['annotation']
        if data_annotations is not None:
            for annotation in data_annotations:
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    point_start = point['start']
                    point_end = point['end']
                    point_text = point['text']

                    lstrip_diff = len(point_text) - len(point_text.lstrip())
                    rstrip_diff = len(point_text) - len(point_text.rstrip())
                    if lstrip_diff != 0:
                        point_start = point_start + lstrip_diff
                    if rstrip_diff != 0:
                        point_end = point_end - rstrip_diff
                    entities.append((point_start, point_end + 1 , label))
        training_data.append((text, {"entities" : entities}))
    return training_data

def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data

In [11]:
data = trim_entity_spans(convert_dataturks_to_spacy(r'C:\Users\razor\Desktop\NLP Assignment\NLPoutput.json'))
data[1]

['Contact www.linkedin.com/in/jia-qing- poh-21a816190 (LinkedIn) Top Skills Critical Thinking Problem Solving Microsoft Excel Languages Mandarin (Full Professional) Malay (Limited Working) English (Native or Bilingual) Jia Qing Poh Statistical Data Science student and aspiring data scientist Kiang Experience AXA Affin Insurance Data Analyst Intern May 2021 - August 2021 (4 months) SOLIDHWUM Vice President November 2019 - May 2020 (7 months) Education Heriot-Watt University Bachelor of Science - BSc, Statistical Data Science * (2019 - 2022) Methodist College Kuala Lumpur A Levels Sekolah Sri Acmar SPM Page 1 of 1',
 {'entities': [[217, 229, 'Name'],
   [453, 475, 'CollegeName'],
   [538, 543, 'GraduationYear'],
   [372, 381, 'YearsofExperience'],
   [308, 327, 'CompaniesWorkAt'],
   [8, 51, 'Email'],
   [74, 123, 'Skills']]}]

In [12]:
def clean_entities(training_data):
    
    clean_data = []
    for text, annotation in training_data:
        
        entities = annotation.get('entities')
        entities_copy = entities.copy()
        
        # append entity only if it is longer than its overlapping entity
        i = 0
        for entity in entities_copy:
            j = 0
            for overlapping_entity in entities_copy:
                # Skip self
                if i != j:
                    e_start, e_end, oe_start, oe_end = entity[0], entity[1], overlapping_entity[0], overlapping_entity[1]
                    # Delete any entity that overlaps, keep if longer
                    if ((e_start >= oe_start and e_start <= oe_end) \
                    or (e_end <= oe_end and e_end >= oe_start)) \
                    and ((e_end - e_start) <= (oe_end - oe_start)):
                        entities.remove(entity)
                j += 1
            i += 1
        clean_data.append((text, {'entities': entities}))
                
    return clean_data

data = clean_entities(data)

In [17]:
data[4]

("Contact www.linkedin.com/in/mohamsup (LinkedIn) Top Skills Microsoft Office Sony Vegas Adobe Premiere Pro Languages Malay (Native or Bilingual) English (Native or Bilingual) Arabic (Limited Working) Bengali (Native or Bilingual) Honors-Awards Dean's List Certificates IIUM Academic Excellence Award IIUM Academic Excellence Award IIUM Ummatic Scholarship UG IIUM Ummatic Scholarship UG Mohammad Bin Yousuf Data Scientist I Data Science I Machine Learning I Artificial intelligence I Big Data Analytics Kuala Lumpur Summary Seeking a Machine Learning Engineer position with a background in Computer Science, focusing on Data Science and Artificial Intelligence. Currently completing a bachelor's in computer science at IIUM. Experienced as a Data Visualization analyst intern at Schlumberger. Experienced in projects related to Data Science, Machine Learning & Cryptocurrency. Acquired additional knowledge in Machine Learning via online certificate courses. Experience Schlumberger Data Visualizati